<a href="https://colab.research.google.com/github/WittmannF/machine-learning-web-app/blob/master/1-scraping/1_b_Coletar_Dados_Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import pandas as pd
import numpy as np
import re
from time import sleep
import requests
from bs4 import BeautifulSoup

In [0]:
# Parametros
ZONAS = ['norte', 'sul', 'leste', 'oeste']
PAGINAS = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']
URL_PADRAO = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'

EXPORTAR_CSV_GDRIVE = False

# Padroes regex
RE_PRECO_M2 = 'R\$ (.*)  (.*) m²'
RE_QUARTO = ' \| (.*) quarto'

In [0]:
# Funcoes
def pegar_codigos_fontes():
    codigo_fontes = []
    for zona in ZONAS:
        for pg in PAGINAS:
            url = URL_PADRAO.format(zona, pg)
            print(f'Carregando URL: {url}')
            codigo_fonte = requests.get(url).content
            codigo_fontes.append(codigo_fonte)
            sleep(3)
    return codigo_fontes

def html_parse(codigo_fontes):
    soups = []
    for source in codigo_fontes:
        soup = BeautifulSoup(source, 'html.parser')
        soups.append(soup)
    return soups

def pegar_resultados(codigo_fontes_bs):
    all_results = []
    for soup in codigo_fontes_bs:
        result = soup.findAll("li", {"class": "results-item highlighted article grid "})
        all_results.extend(result)
    return all_results

def extrair_atributos(all_results):
    n_per_zonas = len(all_results)//len(ZONAS)

    all_zones = []

    for z in ZONAS:
        all_zones.extend([z]*n_per_zonas)

    dados = {'zona': [], 'quartos': [], 'area': [], 'preco': []}

    for r, zona in zip(all_results, all_zones):
        try:
            preco, area = re.findall(RE_PRECO_M2, r.text)[0]
            n_quartos = re.findall(RE_QUARTO, r.text)
            if n_quartos == []:
                n_quartos = 'NaN'
            else:
                n_quartos = n_quartos[0]
            
            dados['zona'].append(zona)
            dados['quartos'].append(n_quartos)
            dados['area'].append(area)
            dados['preco'].append(preco)
        except Exception as e:
            print("Erro, pulando elemento.")
            print("O seguinte erro foi retornado: ")
            print(e)
            
    df = pd.DataFrame(dados)
    return df

def converter_tipos(df):
    df['quartos'] = df['quartos'].astype(float)
    df['area'] = df['area'].astype(float)
    df['preco'] = df['preco'].apply(lambda x: float(x.replace('.', '')))
    return df

In [4]:
# Execucao

# 1. Pegar codigos fontes
codigos_fontes = pegar_codigos_fontes()

Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/
Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/
Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Carregando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Carregando URL: https://imoveis.mercadolivre.com.br/casas/

In [0]:
# 2. Converter para Parser do Beautiful Soup
codigo_fontes_bs = html_parse(codigos_fontes)

In [0]:
# 3. Pegar resultados
all_results = pegar_resultados(codigo_fontes_bs)

In [0]:
# 4. Extrair atributos
df = extrair_atributos(all_results)

In [0]:
# 5. Converter tipos de atributos
df = converter_tipos(df)

In [9]:
# 6. Visualizar base de dados importada
# 6.1 Cabeçaço
df.head()

,zona,quartos,area,preco
0,norte,2.0,90.0,1100.0
1,norte,1.0,50.0,700.0
2,norte,1.0,70.0,600.0
3,norte,1.0,40.0,750.0
4,norte,2.0,90.0,1200.0


In [10]:
# 6.2 Estatísticas
df.describe()

,quartos,area,preco
count,1083.000000,1152.000000,1.152000e+03
mean,2.621422,187.568576,2.117958e+04
std,1.581660,334.020693,2.566756e+05
min,1.000000,10.000000,3.000000e+02
25%,2.000000,60.000000,1.115000e+03
50%,2.000000,125.000000,2.400000e+03
75%,3.000000,240.000000,6.000000e+03
max,17.000000,10000.000000,6.900000e+06


In [0]:
# 7. (Opcional) Montar Google Drive
import sys
if EXPORTAR_CSV_GDRIVE and 'google.colab' in sys.modules:
    try:
        import os
        os.system("pip install easycolab")
        import easycolab as ec
        ec.mount()
    except Exception as e:
        print(e)

In [0]:
# 8. Exportar para CSV
df.to_csv('dados_calculadora_imoveis.csv', index=False)

Se estiver usando o Google Colab, para exportar a base de dados:
 1. Clicar no ícone da pasta na lateral esquerda
 2. Clicar com o botão direito na base de dados
 3. Download

![Screen Shot 2020-03-16 at 15 10 13](https://user-images.githubusercontent.com/5733246/76787874-97576b00-6798-11ea-82f7-a3228bd0a002.png)

